<a href="https://colab.research.google.com/github/Iambusayor/EfficientNLP-workshop/blob/main/Efficiency_in_African_NLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Efficiency in African NLP
todo;
- [ ] Make the notebook readable and properly documented.


## Setup

In [1]:
!git clone https://github.com/Iambusayor/EfficientNLP-workshop.git
%cd EfficientNLP-workshop

Cloning into 'EfficientNLP-workshop'...
remote: Enumerating objects: 37, done.
remote: Counting objects: 100% (37/37), done.
remote: Compressing objects: 100% (27/27), done.
remote: Total 37 (delta 17), reused 24 (delta 8), pack-reused 0
Receiving objects: 100% (37/37), 18.75 KiB | 18.75 MiB/s, done.
Resolving deltas: 100% (17/17), done.
/content/EfficientNLP-workshop


In [2]:
!pip install transformers seqeval ptvsd wandb sentencepiece datasets -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 63.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 4.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 92.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 75.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 75.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.3/519.3 kB 34.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 27.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 52.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 80.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.5/188.5 kB 21.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 218.8/218.8 kB 24.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive



Todo;
---


- [ ] Modify scripts to log metrics on WandB


### WandB Setup
For experiment tracking, all metrics and settings are saved for later reference.

todo;
- [ ] Setup WandB project

In [ ]:
import wandb
wandb.login()

In [4]:
#@title Setting up Environment Variables

language_code = "luo" #@param {type:"string"}
model_name = "bonadossou/afrolm_active_learning" #@param ["Davlan/afro-xlmr-base", "castorini/afriberta_base", "bonadossou/afrolm_active_learning"]

import os
os.environ["language_code"] = language_code
os.environ["MODEL_NAME"]= model_name

os.environ["MODEL_TYPE"] = "xlmroberta" if model_name=="castorini/afriberta-base" else "xlmroberta"

In [5]:
#@title NER Training Configs

%env MAX_LENGTH=164
os.environ["OUTPUT_DIR"] =f"{language_code}_ner"
%env BATCH_SIZE=32
%env NUM_EPOCHS=20
%env SAVE_STEPS=10000
%env SEED=1
!echo "$OUTPUT_DIR"
!echo "$model_name"

env: MAX_LENGTH=164
env: BATCH_SIZE=32
env: NUM_EPOCHS=20
env: SAVE_STEPS=10000
env: SEED=1
luo_ner
bonadossou/afrolm_active_learning


## Dataset

todo;
- [ ] Finish data preprocessing script to incoporate B and I preffix for the labels

In [6]:
#@title Upload Dataset
from google.colab import files
import zipfile
import os
import shutil

#Data directory
data_dir = './data'
os.makedirs(data_dir, exist_ok=True)

# Upload a zip file using the Colab file uploader
uploaded = files.upload()

# Unzip and renanme uploaded file
if uploaded:
    uploaded_filename = list(uploaded.keys())[0]
    print(f"Uploaded zip file: {uploaded_filename}")


    # Unzip the uploaded file
    with zipfile.ZipFile(uploaded_filename, 'r') as zip_ref:
        extracted_content = zip_ref.namelist()  # Get the name of the extracted content
        zip_ref.extractall()

    # Rename extracted files
    filepath = extracted_content[0]
    !python extract_annotations.py --input_path "$filepath" --output_path "$data_dir"
    print(f"{extracted_content[0]} has been processed and written into {data_dir}")
    # Remove redundant files
    os.remove(uploaded_filename)
    shutil.rmtree(extracted_content[0].split('/')[0])
    print(f"Removed redundant files and folders: {uploaded_filename} and {extracted_content[0].split('/')[0]}")
else:
    print("Upload Annotated file")



Saving pigin.zip to pigin.zip
Uploaded zip file: pigin.zip
Writing data into ./data/train.txt: 100% 98/98 [00:00<00:00, 42705.64it/s]
c073db10-4163-11ee-87ac-a7b8e54e4979/export.json has been processed and written into ./data
Removed redundant files and folders: pigin.zip and c073db10-4163-11ee-87ac-a7b8e54e4979


In [45]:
from datasets import load_dataset

# data = load_dataset('masakhane/masakhaner2', language_code)
data = load_dataset('masakhaner', language_code)

flat_dataset = data.flatten()
label_list = data["train"].features["ner_tags"].feature.names

for split in flat_dataset.keys():
    if split != "train":
        split_strings = []
        examples = flat_dataset[split]
        for example in examples:
            words = example["tokens"]
            labels = example["ner_tags"]

            sentence_strings = []
            for word, label in zip(words, labels):
                sentence_strings.append(f"{word} {label_list[label]}")
            split_strings.append("\n".join(sentence_strings))

        output_file_path = os.path.join('./data', f"{split}.txt")
        with open(output_file_path, "w", encoding="utf-8") as spf:
            spf.writelines("\n\n".join(split_strings))

        print(f"{split} data saved to {output_file_path}")


validation data saved to ./data/validation.txt
test data saved to ./data/test.txt


In [43]:
data

DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'ner_tags'],
        num_rows: 644
    })
    validation: Dataset({
        features: ['id', 'tokens', 'ner_tags'],
        num_rows: 92
    })
    test: Dataset({
        features: ['id', 'tokens', 'ner_tags'],
        num_rows: 186
    })
})

In [8]:
os.rename('./data/validation.txt', './data/dev.txt')
!ls ./data

dev.txt  test.txt  train.txt


In [9]:
ner_feature = data["train"].features["ner_tags"]
ner_feature.feature.names

['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-DATE', 'I-DATE']

In [10]:
!ls -la

total 88
drwxr-xr-x 4 root root  4096 Aug 30 09:26 .
drwxr-xr-x 1 root root  4096 Aug 30 09:26 ..
drwxr-xr-x 2 root root  4096 Aug 30 09:26 data
-rw-r--r-- 1 root root  2440 Aug 30 09:26 extract_annotations.py
-rw-r--r-- 1 root root   269 Aug 30 09:26 extract.sh
drwxr-xr-x 8 root root  4096 Aug 30 09:26 .git
-rw-r--r-- 1 root root  3078 Aug 30 09:26 .gitignore
-rw-r--r-- 1 root root    23 Aug 30 09:26 README.md
-rw-r--r-- 1 root root     0 Aug 30 09:26 requirements.txt
-rw-r--r-- 1 root root   739 Aug 30 09:26 run.sh
-rw-r--r-- 1 root root   886 Aug 30 09:26 train_ner.ipynb
-rw-r--r-- 1 root root 33109 Aug 30 09:26 train_ner.py
-rw-r--r-- 1 root root  9621 Aug 30 09:26 utils_ner.py


In [11]:
!ls ./data

dev.txt  test.txt  train.txt


## Training;
**Fine-tune Model on Data and evaluate on MasakhaNER 2.0**

In [17]:
os.environ["MAX_LENGTH"] = "164"
os.environ["OUTPUT_DIR"] = f"{model_name}_masakhaner"
os.environ["TEXT_RESULT"] = "test_result.txt"
os.environ["TEXT_PREDICTION"] = "test_predictions.txt"
os.environ["BATCH_SIZE"] = "32"
os.environ["NUM_EPOCHS"] = "50"
os.environ["SAVE_STEPS"] = "10000"
os.environ["SEED"] = "1"
os.environ["DATA_DIR"] = "./data"


!CUDA_VISIBLE_DEVICES=0,1 python3 train_ner.py --data_dir $DATA_DIR \
--model_type $MODEL_TYPE \
--model_name_or_path $MODEL_NAME \
--output_dir $OUTPUT_DIR \
--test_result_file $TEXT_RESULT \
--test_prediction_file $TEXT_PREDICTION \
--max_seq_length  $MAX_LENGTH \
--num_train_epochs $NUM_EPOCHS \
--per_gpu_train_batch_size $BATCH_SIZE \
--save_steps $SAVE_STEPS \
--seed $SEED \
--do_train \
--do_eval \
--do_predict \
--overwrite_output_dir


2023-08-30 09:45:37.387403: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Some weights of XLMRobertaForTokenClassification were not initialized from the model checkpoint at bonadossou/afrolm_active_learning and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
08/30/2023 09:45:58 - INFO - __main__ -   Training/evaluation parameters Namespace(data_dir='./data', model_type='xlmroberta', model_name_or_path='bonadossou/afrolm_active_learning', input_dir=None, output_dir='bonadossou/afrolm_active_learning_masakhaner', test_result_file='test_result.txt', test_prediction_file='test_predictions.txt', labels='', config_name='', tokenizer_name='', cache_dir='', max_seq_length=164, do_train=True, do_finetune=False, do_eval=True, do_predict=True, evaluate_during_training=False, do_lower_case=False, per_gpu_train_batch_size=32

## Backup Data and Model

In [7]:
author_name = input("Please enter your name:\t")
os.makedirs(f'backup_folder_name /content/drive/masakhane/Efficiency-workshop/{language_code}_{author_name}', exist_ok=True)

Please enter your nameBusayor


In [18]:
!cp -rv ./* "$backup_folder_name"

cp: target '' is not a directory
